In [1]:
filename = "train_labels.csv"
import pandas as pd 
df = pd.read_csv(filename, encoding='utf-8')
df.head()

,label
0,normal
1,normal
2,neptune
3,normal
4,normal


In [2]:
df["images"]=""
from tqdm import tqdm 
for i in tqdm(range(len(df))):
    if i == 0: 
        continue
    df["images"][i] = str(i)+".jpg"
print(df.head())

100%|██████████| 125973/125973 [01:16<00:00, 1461.11it/s]


     label images
0   normal       
1   normal  1.jpg
2  neptune  2.jpg
3   normal  3.jpg
4   normal  4.jpg


In [23]:
df.tail() 

,label,images
125968,neptune,125969.jpg
125969,normal,125970.jpg
125970,normal,125971.jpg
125971,neptune,125972.jpg
125972,normal,125973.jpg


In [3]:
imagefiles_list = []
from PIL import Image
import numpy as np

for i in tqdm(range(len(df))):
    if i == 0: 
        continue
    imagefiles_list.append(df['images'][i])
    
size = (8,8)
x = np.array([np.array(Image.open("./img_data_rev/img_data/"+fname).resize(size).convert('RGB')) for fname in tqdm(imagefiles_list)])
print(x.shape)

x.dump("train_features.pkl")

100%|██████████| 125972/125972 [05:27<00:00, 439.17it/s]


(125972, 8, 8, 3)


In [5]:
df["label"][1:]

1              normal
2             neptune
3              normal
4              normal
5             neptune
6             neptune
7             neptune
8             neptune
9             neptune
10            neptune
11            neptune
12             normal
13        warezclient
14            neptune
15            neptune
16             normal
17            ipsweep
18             normal
19             normal
20            neptune
21            neptune
22             normal
23             normal
24            neptune
25             normal
26            neptune
27             normal
28             normal
29             normal
30            ipsweep
             ...     
125943         normal
125944         normal
125945         normal
125946        neptune
125947        ipsweep
125948        neptune
125949         normal
125950       teardrop
125951         normal
125952         normal
125953        neptune
125954         normal
125955         normal
125956         normal
125957    

In [6]:
labels = pd.get_dummies(df["label"][1:]).as_matrix()

print(labels.shape)
labels.dump("train_labels.pkl")

(125972, 23)


In [9]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv2D,MaxPooling2D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
import time
from keras.utils import np_utils
from sklearn import preprocessing as pp

import h5py

X_train = np.load("train_features.pkl")
Y_train = np.load("train_labels.pkl")



cnn2d_1 = Sequential([Conv2D(64, (3,3), padding="same",input_shape=(8,8,3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(23),
    Activation('sigmoid'),
])
print(cnn2d_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 8, 8, 64)          1792      
_________________________________________________________________
activation_7 (Activation)    (None, 8, 8, 64)          0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               131200    
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [13]:
test_df = pd.read_csv("test_labels.csv", encoding='utf-8')
test_df["images"]=""
from tqdm import tqdm 
for i in tqdm(range(len(test_df))):
    if i == 0: 
        continue
    test_df["images"][i] = str(i)+".jpg"
test_imagefiles_list = []
for i in tqdm(range(len(test_df))):
    if i == 0: 
        continue 
    test_imagefiles_list.append(test_df['images'][i])
    
size = (8,8)
test_x = np.array([np.array(Image.open("./img_kddtest+/img_kddtest+/"+fname).resize(size).convert('RGB')) for fname in tqdm(test_imagefiles_list)])
print(test_x.shape)

test_x.dump("test_features.pkl")

test_labels = pd.get_dummies(test_df["label"]).as_matrix()

print(test_labels.shape)
test_labels.dump("test_labels.pkl")


100%|██████████| 22543/22543 [00:05<00:00, 3806.05it/s]

(22543, 8, 8, 3)
(22544, 38)


In [16]:
import time
cnn2d_1.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn2d_1.fit(X_train, Y_train, batch_size=64,epochs=10)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn2d_1.to_json()
with open("NSL_KDD_Train.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("NSL_KDD_Train_weights.h5")

Epoch 1/10
125972/125972 [==============================] - 13s 100us/step - loss: 2.4671 - acc: 0.0304
Epoch 2/10
125972/125972 [==============================] - 12s 99us/step - loss: 2.4718 - acc: 0.0460
Epoch 3/10
125972/125972 [==============================] - 12s 98us/step - loss: 2.6962 - acc: 0.2469
Epoch 4/10
125972/125972 [==============================] - 12s 97us/step - loss: 2.8235 - acc: 0.3138
Epoch 5/10
125972/125972 [==============================] - 12s 96us/step - loss: 2.8262 - acc: 0.3261
Epoch 6/10
125972/125972 [==============================] - 12s 96us/step - loss: 2.8267 - acc: 0.3269
Epoch 7/10
125972/125972 [==============================] - 12s 96us/step - loss: 2.8267 - acc: 0.3268
Epoch 8/10
125972/125972 [==============================] - 12s 98us/step - loss: 2.8268 - acc: 0.3269
Epoch 9/10
125972/125972 [==============================] - 12s 99us/step - loss: 2.8267 - acc: 0.3268
Epoch 10/10
125972/125972 [==============================] - 12s 98us/st

NameError: name 'cnn1d_1' is not defined

In [ ]:
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_Valaccuaracy00_01.png')